In [354]:
import matplotlib.pyplot as plt
import pandas as pd

# 1
Utilize o conjunto de dados [Go To College Dataset](https://www.kaggle.com/datasets/saddamazyazy/go-to-college-dataset) ou um conjunto de dados de **classificação supervisionada** de sua escolha.

In [355]:
df_gotocollege = pd.read_csv('GoToCollege.csv')

df = df_gotocollege.copy()

df.head()

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,will_go_to_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,False,True
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,False,True
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,False,True
3,Vocational,B,Male,Very Interested,Rural,49,6600000,78.2,82.12,True,True
4,Academic,A,Female,Very Interested,Urban,57,5250000,75.1,86.79,False,False


In [356]:
for i in df['type_school']:
    type_school = df['type_school'].replace(['Academic','Vocational'],['0','1'])
    df['type_school'] = type_school

for i in df['school_accreditation']:
    school_accreditation = df['school_accreditation'].replace(['A','B'],['0','1'])
    df['school_accreditation'] = school_accreditation

for i in df['gender']:
    gender = df['gender'].replace(['Male','Female'],['0','1'])
    df['gender'] = gender

for i in  df['interest']:
    interest = df['interest'].replace(['Uncertain','Not Interested','Interested','Less Interested','Very Interested'], ['0', '1', '2', '3','4'])
    df['interest'] = interest

for i in df['residence']:
    residence = df['residence'].replace(['Rural','Urban'], ['0','1'])
    df['residence'] = residence
                                              
df['parent_was_in_college']  = df['parent_was_in_college'].map({True:1, False:0})
df['will_go_to_college']  = df['will_go_to_college'].map({True:1, False:0})

In [357]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   int64  
 10  will_go_to_college     1000 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 86.1+ KB


# 2
Utilize a técnica k-fold cross validation. Compare os resultados com o melhor resultado da Atividade 6.

Validação cruzada é um método que permite avaliar a performance preditiva de um modelo de previsão.

In [358]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X = df.drop(columns=['will_go_to_college'])
y = df['will_go_to_college'].values

knn = KNeighborsClassifier(n_neighbors = 11)
print('KNN com 11 vizinhos')
print('-----------------------------------------------------------------------------')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1, stratify=y) #50/50
fit = knn.fit(X_train, y_train)
predict = knn.predict(X_test)
acc_HO = accuracy_score(y_test, predict)
print('Hold-Out (train = 50, test = 50), Acuracia:',acc_HO)

scores = cross_val_score(knn, X, y, cv=3, scoring='accuracy')
print('K-Fold Cross Validation (K = 3), Acuracia:', scores)

KNN com 11 vizinhos
-----------------------------------------------------------------------------
Hold-Out (train = 50, test = 50), Acuracia: 0.712
K-Fold Cross Validation (K = 3), Acuracia: [0.69760479 0.74774775 0.68768769]


# 3
Realize experimentos utilizando normalização e padronização. Compare os resultados com o melhor resultado da Atividade 6.

# Normalização

In [359]:
from sklearn.preprocessing import MinMaxScaler

scaler_norm = MinMaxScaler()
# transform data
scaled_norm = scaler_norm.fit_transform(df)
print(scaled_norm)


[[0.         0.         0.         ... 0.39521739 0.         1.        ]
 [0.         0.         0.         ... 0.51782609 0.         1.        ]
 [0.         1.         1.         ... 0.54043478 0.         1.        ]
 ...
 [1.         0.         0.         ... 0.19826087 0.         1.        ]
 [0.         1.         0.         ... 0.52956522 1.         1.        ]
 [0.         1.         0.         ... 0.48391304 1.         0.        ]]


# Padronização

In [360]:
from sklearn.preprocessing import StandardScaler

scaler_padr = StandardScaler()
# transform data
scaled_padr = scaler_padr.fit_transform(df)
print(scaled_padr)

[[-0.80127157 -1.03875025 -0.97043679 ... -0.59436515 -1.040833
   1.        ]
 [-0.80127157 -1.03875025 -0.97043679 ...  0.24068354 -1.040833
   1.        ]
 [-0.80127157  0.96269532  1.03046381 ...  0.39466414 -1.040833
   1.        ]
 ...
 [ 1.24801633 -1.03875025 -0.97043679 ... -1.93577314 -1.040833
   1.        ]
 [-0.80127157  0.96269532 -0.97043679 ...  0.320635    0.96076892
   1.        ]
 [-0.80127157  0.96269532 -0.97043679 ...  0.00971262  0.96076892
  -1.        ]]


# Comparando resultados

In [361]:
from sklearn.datasets import load_wine
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

knn_pipeline = Pipeline(steps=[
  ("normalizacao", MinMaxScaler()),  
  ("KNN", KNeighborsClassifier(n_neighbors=11))]
  )

In [362]:
from sklearn.model_selection import train_test_split
import numpy as np
N = 5
acuracias = []
scores_ = []

for i in range(N):
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.5, shuffle=True)
    knn_pipeline.fit(X_treino, y_treino)
    ac_i = knn_pipeline.score(X_teste, y_teste)
    acuracias.append(ac_i)

for i in range(N):
    scores = cross_val_score(knn_pipeline, X, y, cv=3, scoring='accuracy')
    scores_.append(scores)

print('-Acuracia (Hold-Out)-')
print(f'Media: {round(np.mean(acuracias) * 100, 2)}%')
print(f'Desvio padrão: {round(np.std(acuracias) * 100, 2)}%')

print('-----------------------------')

print('-Acuracia (Cross-Validation)-')
print(f'Media: {round(np.mean(scores_) * 100, 2)}%')
print(f'Desvio padrão: {round(np.std(scores_) * 100, 2)}%')

-Acuracia (Hold-Out)-
Media: 77.92%
Desvio padrão: 0.61%
-----------------------------
-Acuracia (Cross-Validation)-
Media: 79.0%
Desvio padrão: 2.41%


Uma maneira comum de se avaliar o desempenho de um modelo treinado a partir de um conjunto de dados é executar o pipeline de treinamento e teste multiplas vezes e calcular a média e desvio padrão da métrica de desempenho escolhida.

# 4
Tente realizar alguma outra etapa pré-processamento/transformação dos dados. Compare os resultados com os resultados já obtidos.

Umas das outras etapas conhecidas: Label Enconding e One Hot Enconding.

In [363]:
from sklearn.preprocessing import LabelEncoder #transforma as colunas categoricas

label_encoder = LabelEncoder()

for i in df.columns:
    label_encoder.fit(df[i])

df


,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,will_go_to_college
0,0,0,0,3,1,56,6950000,83.0,84.09,0,1
1,0,0,0,3,1,57,4410000,76.8,86.91,0,1
2,0,1,1,4,1,50,6500000,80.6,87.43,0,1
3,1,1,0,4,0,49,6600000,78.2,82.12,1,1
4,0,0,1,4,1,57,5250000,75.1,86.79,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1,0,1,4,0,49,7420000,63.6,85.99,1,1
996,0,1,1,3,0,51,7480000,84.3,89.72,1,1
997,1,0,0,3,1,49,5550000,75.2,79.56,0,1
998,0,1,0,0,0,53,5840000,105.8,87.18,1,1


# Nota de estudo

O método hold-out é bom para quando você tem um conjunto de dados muito grande, está com pouco tempo ou está começando a criar um modelo inicial em ciência de dados. Lembrando que, como a validação cruzada (Cross validation) usa várias divisões de teste de treinamento, é preciso mais poder computacional e tempo para executar do que usar o método de validação.